In [1]:
client_id = 
client_secret = 

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 2im35h7lxxbzynyi0cl5lfyi3j4yb5


In [2]:
oauth_token = '2im35h7lxxbzynyi0cl5lfyi3j4yb5'

In [3]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/thegrefg"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '48878319', 'login': 'thegrefg', 'display_name': 'TheGrefg', 'type': '', 'broadcaster_type': 'partner', 'description': 'Hola, me llamo David aunque la gente me llama "TheGrefg" y me gusta crear contenido con PASIÓN =) Business: contact@thegrefg.com', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/96fef5c0-9ec8-4fd3-a360-90b589c3ff0d-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/5043eda4-570c-444e-8e53-204caf0803cc-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2013-09-12T00:45:27Z'}]}


In [4]:
data = {'data': [{'id': '48878319', 'login': 'thegrefg', 'display_name': 'TheGrefg', 'type': '', 'broadcaster_type': 'partner', 'description': 'Hola, me llamo David aunque la gente me llama "TheGrefg" y me gusta crear contenido con PASIÓN =) Business: contact@thegrefg.com', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/96fef5c0-9ec8-4fd3-a360-90b589c3ff0d-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/5043eda4-570c-444e-8e53-204caf0803cc-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2013-09-12T00:45:27Z'}]}
data_dict = data['data'][0]
data_dict
id_canal =  list(data_dict.values())[0]
id_canal

'48878319'

In [5]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             62 non-null     int64         
 1   URL del VOD            62 non-null     object        
 2   Nombre del VOD         62 non-null     object        
 3   Duración               62 non-null     object        
 4   Fecha de inicio        62 non-null     datetime64[ns]
 5   Fecha de finalización  62 non-null     datetime64[ns]
 6   Visitas                62 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 38.9 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1890896278,https://www.twitch.tv/videos/1890896278,😍_𝐍𝐎𝐒𝐓𝐀𝐋𝐆𝐈-𝐂𝐎𝐃𝐬_😍_zombies_de_black_ops_1_¿east...,4h12m15s,2023-08-05 16:34:29,2023-08-05 16:34:29,336883
1,1890001980,https://www.twitch.tv/videos/1890001980,✨_𝐍𝐎𝐒𝐓𝐀𝐋𝐆𝐈-𝐂𝐎𝐃𝐬_✨_black_ops_1_😱_zombies_y_mult...,3h37m40s,2023-08-04 16:30:01,2023-08-04 16:30:01,249455
2,1889213315,https://www.twitch.tv/videos/1889213315,✨_𝐍𝐎𝐒𝐓𝐀𝐋𝐆𝐈-𝐂𝐎𝐃𝐬_✨_me_saco_la_nuclear_🔥_black_o...,5h5m41s,2023-08-03 18:16:32,2023-08-03 18:16:32,394961
3,1888230819,https://www.twitch.tv/videos/1888230819,✨_𝐍𝐎𝐒𝐓𝐀𝐋𝐆𝐈𝐀_✨_volvemos_a_los_cods_antiguos_😍_1...,5h35m53s,2023-08-02 15:45:33,2023-08-02 15:45:33,579052
4,1887533185,https://www.twitch.tv/videos/1887533185,❤🔥_he_vinilado_mi_tesla_de_rayo_mcqueen_🔥❤_|_m...,1h40m10s,2023-08-01 19:14:42,2023-08-01 19:14:42,179403


In [6]:
df.to_csv('twitch_api_thegrefg_csv.csv', index=False)